In [1]:
%reset -f

In [2]:
import pandas as pd
from sssom.parsers import parse_sssom_table
from sssom import compare_dataframes
from sssom.parsers import split_dataframe
from sssom.util import MappingSetDataFrame
from os.path import join
from oaklib import OntologyResource
from oaklib.implementations import SqlImplementation
import textdistance

In [3]:
lexmatch_file = "../mappings/mondo-sources-all-lexical.sssom.tsv"
mondo_sssom = "../ontology/tmp/mondo.sssom.tsv"
dir_name = "dataframes"

In [4]:
# Functions

def add_distance(df, col_name, txt_dist_pkg):
    df\
    .insert(\
            len(df.columns),\
            col_name,\
            df.apply\
            (\
                lambda x: txt_dist_pkg\
                 (\
                    x.subject_label.lower(), x.object_label.lower()\
                    if pd.notnull(x.object_label) else "99"\
                 ),\
                axis=1,\
            )\
           )

def print_prefixes(df):
    object_prefixes = df['object_id'].str.split(':').apply(lambda x: x[0] ).drop_duplicates()
    subject_prefixes = df['subject_id'].str.split(':').apply(lambda x: x[0] ).drop_duplicates()
    predicate_ids = df['predicate_id'].drop_duplicates()

    print(f"subject_prefixes:\n {subject_prefixes} \n \
          object_prefixes:\n {object_prefixes} \n \
          predicate_ids:\n {predicate_ids} ")

    
def flip_predicate(predicate_id):
    flip_dict = {
        "skos:closeMatch": "skos:relatedMatch",
        "skos:relatedMatch": "skos:closeMatch",
        "skos:narrowMatch" : "skos:broadMatch",
        "skos:broadMatch" : "skos:narrowMatch",
        "skos:exactMatch" : "skos:exactMatch"
    }
    
    return flip_dict[predicate_id]

def compare_and_comment_df(mondo_df, lex_df):
    df = compare_dataframes(mondo_df, lex_df).combined_dataframe
    df['comment'] = df['comment'].str.replace('UNIQUE_1', "MONDO_MAPPINGS")
    df['comment'] = df['comment'].str.replace('UNIQUE_2', "LEXMATCH")
    return df

def get_unmapped_df(comparison_df):
#     mappings = ["LEXMATCH", "MONDO_MAPPINGS"]
#     unmapped_df = comparison_df[
#         (comparison_df['comment'].str.contains("|".join(mappings)))
#     ]
    unmapped_lex_df = comparison_df[
        comparison_df['object_id'].str.contains("|".join(in_lex_but_not_mondo_list)) &
        comparison_df['comment'].str.contains("LEXMATCH")
    ]
    
    unmapped_mondo_df = comparison_df[
        comparison_df['object_id'].str.contains("|".join(in_mondo_but_not_lex_list)) &
        comparison_df['comment'].str.contains("MONDO_MAPPINGS")
    ]
    
    new_df = pd.concat([unmapped_lex_df, unmapped_mondo_df], axis=0)
    return new_df

def export_unmatched_exact(unmapped_df, match_type, fn):
        
    unmapped_exact = unmapped_df[(unmapped_df['comment'] == match_type) & (unmapped_df['predicate_id'] == 'skos:exactMatch')]
    unmapped_exact.to_csv(join(dir_name, fn), sep='\t', index = False)
    return unmapped_exact.head()

def make_msdf(comparison_df, prefix_map, meta):
    combined_msdf = MappingSetDataFrame(df=comparison_df, prefix_map=prefix_map, metadata=meta)
    return combined_msdf

def mapped_curie_list(df):
    all_id_df = pd.DataFrame(pd.concat([df['subject_id'], df['object_id']]))\
                .drop_duplicates(ignore_index=True)
    all_id_df = all_id_df[~all_id_df[0].str.startswith('MONDO')]
    return all_id_df[0].to_list()


In [5]:
%%time
msdf_lex = parse_sssom_table(lexmatch_file)
msdf_mondo = parse_sssom_table(mondo_sssom)

# Use OAK to get 'object_label'
ontology_resource = OntologyResource(slug='../ontology/tmp/merged.db', local=True)
oi = SqlImplementation(ontology_resource)

CPU times: user 3min 15s, sys: 2.46 s, total: 3min 17s
Wall time: 3min 19s


In [6]:
%%time
all_lex_ids = mapped_curie_list(msdf_lex.df)
all_mondo_ids = mapped_curie_list(msdf_mondo.df)

in_lex_but_not_mondo_list = [x for x in all_lex_ids if x not in all_mondo_ids]
in_mondo_but_not_lex_list = [x for x in all_mondo_ids if x not in all_lex_ids]


CPU times: user 4min 33s, sys: 274 ms, total: 4min 33s
Wall time: 4min 33s


In [7]:
in_lex_but_not_mondo_list

['DOID:0040046',
 'DOID:0040101',
 'DOID:0060042',
 'DOID:0060160',
 'DOID:0060204',
 'DOID:0070020',
 'DOID:0070141',
 'DOID:0070212',
 'DOID:0070309',
 'DOID:0070311',
 'DOID:0070326',
 'DOID:0070327',
 'DOID:0070331',
 'DOID:0070333',
 'DOID:0070341',
 'DOID:0070343',
 'DOID:0070348',
 'DOID:0070349',
 'DOID:0070351',
 'DOID:0070358',
 'DOID:0070359',
 'DOID:0070360',
 'DOID:0070361',
 'DOID:0070362',
 'DOID:0080380',
 'DOID:0080390',
 'DOID:0080407',
 'DOID:0080410',
 'DOID:0080411',
 'DOID:0080503',
 'DOID:0080511',
 'DOID:0080546',
 'DOID:0080574',
 'DOID:0080594',
 'DOID:0080597',
 'DOID:0080602',
 'DOID:0080606',
 'DOID:0080607',
 'DOID:0080610',
 'DOID:0080612',
 'DOID:0080638',
 'DOID:0080641',
 'DOID:0080644',
 'DOID:0080645',
 'DOID:0080646',
 'DOID:0080647',
 'DOID:0080650',
 'DOID:0080651',
 'DOID:0080652',
 'DOID:0080661',
 'DOID:0080662',
 'DOID:0080663',
 'DOID:0080664',
 'DOID:0080665',
 'DOID:0080666',
 'DOID:0080669',
 'DOID:0080670',
 'DOID:0080671',
 'DOID:0080672

In [8]:
print_prefixes(msdf_mondo.df)

subject_prefixes:
 0    MONDO
Name: subject_id, dtype: object 
           object_prefixes:
 0           SCTID
1            MESH
2            DOID
3        Orphanet
4            NCIT
5            UMLS
11         OMIMPS
51        ICD10CM
172          OMIM
604        MedDRA
657      ICD10WHO
25506      MEDGEN
Name: object_id, dtype: object 
           predicate_ids:
 0      skos:exactMatch
111    skos:broadMatch
Name: predicate_id, dtype: object 


In [9]:
%%time
# msdf_mondo.df[msdf_mondo.df['object_id'].str.contains('ICD')]
# "ICD10CM", "MONDO"
# msdf_mondo.df = msdf_mondo.df[(condition_1 & condition_2) | (condition_3 & condition_4)]
# msdf_mondo.df['object_label'] = msdf_mondo.df['object_id'].apply(lambda x: oi.label(x))

msdf_mondo.df['object_label'] = msdf_mondo.df['object_id'].apply(lambda x: oi.label(x))
msdf_mondo.df.head()


CPU times: user 1min 13s, sys: 5.19 s, total: 1min 19s
Wall time: 1min 19s


,subject_id,subject_label,predicate_id,object_id,mapping_justification,object_label
0,MONDO:0000001,disease or disorder,skos:exactMatch,SCTID:64572001,Unspecified,None
1,MONDO:0000001,disease or disorder,skos:exactMatch,MESH:D004194,Unspecified,None
2,MONDO:0000001,disease or disorder,skos:exactMatch,DOID:4,Unspecified,disease
3,MONDO:0000001,disease or disorder,skos:exactMatch,Orphanet:377788,Unspecified,Disease
4,MONDO:0000001,disease or disorder,skos:exactMatch,NCIT:C2991,Unspecified,Disease or Disorder


In [10]:
condition_1 = msdf_mondo.df['subject_id'].str.contains("MONDO")
condition_2 = msdf_mondo.df['object_id'].str.contains("ICD10CM")
condition_3 = msdf_mondo.df['object_id'].str.contains('|'.join((["OMIM","OMIMPS"])))
condition_4 = msdf_mondo.df['object_id'].str.contains("Orphanet")
condition_5 = msdf_mondo.df['object_id'].str.contains("DOID")

mondo_icd_df = msdf_mondo.df[condition_1 & condition_2]
mondo_omim_df = msdf_mondo.df[condition_1 & condition_3]
mondo_ordo_df = msdf_mondo.df[condition_1 & condition_4]
mondo_doid_df = msdf_mondo.df[condition_1 & condition_5]

mondo_icd_df.head()
mondo_omim_df.head()
mondo_ordo_df.head()
# mondo_doid_df.head()


,subject_id,subject_label,predicate_id,object_id,mapping_justification,object_label
3,MONDO:0000001,disease or disorder,skos:exactMatch,Orphanet:377788,Unspecified,Disease
20,MONDO:0000023,infantile liver failure,skos:exactMatch,Orphanet:464724,Unspecified,Fever-associated acute infantile liver failure...
26,MONDO:0000044,hereditary hypophosphatemic rickets,skos:exactMatch,Orphanet:437,Unspecified,Hypophosphatemic rickets
33,MONDO:0000050,isolated congenital growth hormone deficiency,skos:exactMatch,Orphanet:631,Unspecified,Non-acquired isolated growth hormone deficiency
49,MONDO:0000087,polymicrogyria,skos:exactMatch,Orphanet:35981,Unspecified,Polymicrogyria


In [11]:
print_prefixes(msdf_lex.df)

subject_prefixes:
 0             DOID
151307     ICD10CM
166651    ICD10WHO
169860       MONDO
425468        NCIT
458795        OMIM
474649    Orphanet
Name: subject_id, dtype: object 
           object_prefixes:
 0         MONDO
7          NCIT
8      Orphanet
11     ICD10WHO
20         DOID
37      ICD10CM
199        OMIM
Name: object_id, dtype: object 
           predicate_ids:
 0       skos:broadMatch
1       skos:exactMatch
18      skos:closeMatch
176    skos:narrowMatch
Name: predicate_id, dtype: object 


### Flipping subject_id and object_id if MONDO not subject_id prefix

**Predicate impact**

- skos:closeMatch <=> skos:relatedMatch
- skos:narrowMatch < => skos:broadMatch

In [12]:
condition_1 = msdf_lex.df['subject_id'].str.contains("MONDO")
condition_2 = msdf_lex.df['object_id'].str.contains("ICD10CM")
condition_3 = msdf_lex.df['object_id'].str.contains('|'.join((["OMIM","OMIMPS"])))
condition_4 = msdf_lex.df['object_id'].str.contains('|'.join((["ORDO","Orphanet"])))
condition_5 = msdf_lex.df['object_id'].str.contains("DOID")
condition_mondo_obj = msdf_lex.df['object_id'].str.contains("MONDO")
non_mondo_subjects_df =  pd.DataFrame(msdf_lex.df[(~condition_1 & condition_mondo_obj)])
mondo_subjects_df = pd.DataFrame(msdf_lex.df[(condition_1 & ~condition_mondo_obj)])
print(len(mondo_subjects_df))
non_mondo_subjects_df.head()


44371


,subject_id,subject_label,predicate_id,object_id,object_label,mapping_justification,mapping_tool,confidence,subject_match_field,object_match_field,match_string
0,DOID:0001816,angiosarcoma,skos:broadMatch,MONDO:0003022,pediatric angiosarcoma,semapv:LexicalMatching,oaklib,0.800000,rdfs:label,oio:hasBroadSynonym,angiosarcoma
1,DOID:0001816,angiosarcoma,skos:exactMatch,MONDO:0016982,angiosarcoma,semapv:LexicalMatching,oaklib,0.941176,oio:hasDbXref,oio:hasDbXref,mesh:d006394
2,DOID:0001816,angiosarcoma,skos:exactMatch,MONDO:0016982,angiosarcoma,semapv:LexicalMatching,oaklib,0.800000,rdfs:label,oio:hasExactSynonym,angiosarcoma
3,DOID:0001816,angiosarcoma,skos:exactMatch,MONDO:0016982,angiosarcoma,semapv:RegularExpressionReplacement,oaklib,0.800000,rdfs:label,oio:hasExactSynonym,angiosarcoma
4,DOID:0001816,angiosarcoma,skos:exactMatch,MONDO:0016982,angiosarcoma,semapv:LexicalMatching,oaklib,0.800000,oio:hasExactSynonym,oio:hasExactSynonym,hemangiosarcoma


In [13]:
desired_sequence = ['subject_id', 'subject_label', 'predicate_id', 'object_id',
       'object_label', 'mapping_justification', 'mapping_tool', 'confidence',
       'subject_match_field', 'object_match_field', 'match_string']

new_subjects_df = non_mondo_subjects_df.rename(columns={
    'subject_id':'object_id', 
    'subject_label':'object_label',
    'object_id':'subject_id', 
    'object_label':'subject_label'
    
})


new_subjects_df = new_subjects_df[desired_sequence]
new_subjects_df["predicate_id"] = new_subjects_df["predicate_id"].apply(lambda x: flip_predicate(x))
print(len(new_subjects_df))
new_subjects_df.head()


103462


,subject_id,subject_label,predicate_id,object_id,object_label,mapping_justification,mapping_tool,confidence,subject_match_field,object_match_field,match_string
0,MONDO:0003022,pediatric angiosarcoma,skos:narrowMatch,DOID:0001816,angiosarcoma,semapv:LexicalMatching,oaklib,0.800000,rdfs:label,oio:hasBroadSynonym,angiosarcoma
1,MONDO:0016982,angiosarcoma,skos:exactMatch,DOID:0001816,angiosarcoma,semapv:LexicalMatching,oaklib,0.941176,oio:hasDbXref,oio:hasDbXref,mesh:d006394
2,MONDO:0016982,angiosarcoma,skos:exactMatch,DOID:0001816,angiosarcoma,semapv:LexicalMatching,oaklib,0.800000,rdfs:label,oio:hasExactSynonym,angiosarcoma
3,MONDO:0016982,angiosarcoma,skos:exactMatch,DOID:0001816,angiosarcoma,semapv:RegularExpressionReplacement,oaklib,0.800000,rdfs:label,oio:hasExactSynonym,angiosarcoma
4,MONDO:0016982,angiosarcoma,skos:exactMatch,DOID:0001816,angiosarcoma,semapv:LexicalMatching,oaklib,0.800000,oio:hasExactSynonym,oio:hasExactSynonym,hemangiosarcoma


### Combine dfs where subject_id prefix is MONDO

In [14]:
lex_df = pd.concat([mondo_subjects_df,new_subjects_df], ignore_index=True)
print(len(lex_df))

lex_df.head()

147833


,subject_id,subject_label,predicate_id,object_id,object_label,mapping_justification,mapping_tool,confidence,subject_match_field,object_match_field,match_string
0,MONDO:0000001,disease or disorder,skos:exactMatch,Orphanet:557493,disorder,semapv:LexicalMatching,oaklib,0.8,oio:hasExactSynonym,rdfs:label,disorder
1,MONDO:0000001,disease or disorder,skos:exactMatch,NCIT:C25457,Condition,semapv:LexicalMatching,oaklib,0.8,oio:hasExactSynonym,rdfs:label,condition
2,MONDO:0000001,disease or disorder,skos:exactMatch,NCIT:C156809,Medical Condition,semapv:LexicalMatching,oaklib,0.8,oio:hasExactSynonym,rdfs:label,medical condition
3,MONDO:0000004,adrenocortical insufficiency,skos:exactMatch,NCIT:C113211,Hypocortisolemia,semapv:LexicalMatching,oaklib,0.8,oio:hasExactSynonym,rdfs:label,hypocortisolemia
4,MONDO:0000022,nocturnal enuresis,skos:exactMatch,OMIM:600631,"enuresis, nocturnal, 1",semapv:LexicalMatching,oaklib,0.8,oio:hasExactSynonym,oio:hasExactSynonym,bedwetting


In [15]:
# msdf_lex.df[msdf_lex.df['object_id'].str.contains('ICD')]
condition_1 = lex_df['subject_id'].str.contains("MONDO")
condition_2 = lex_df['object_id'].str.contains("ICD10CM")
condition_3 = lex_df['object_id'].str.contains('|'.join((["OMIM","OMIMPS"])))
condition_4 = lex_df['object_id'].str.contains("Orphanet")
condition_5 = lex_df['object_id'].str.contains("DOID")


mondo_icd_lex_df = lex_df[(condition_1 & condition_2)]
mondo_omim_lex_df = lex_df[(condition_1 & condition_3)]
mondo_ordo_lex_df = lex_df[(condition_1 & condition_4)]
mondo_doid_lex_df = lex_df[(condition_1 & condition_5)]

mondo_icd_lex_df.head()
mondo_omim_lex_df.head() # NO ROWS
mondo_ordo_lex_df.head() # NO ROWS
mondo_doid_lex_df.head()


,subject_id,subject_label,predicate_id,object_id,object_label,mapping_justification,mapping_tool,confidence,subject_match_field,object_match_field,match_string
968,MONDO:0004990,breast tumor luminal A or B,skos:narrowMatch,DOID:0060548,luminal breast carcinoma A,semapv:LexicalMatching,oaklib,0.800000,oio:hasExactSynonym,oio:hasBroadSynonym,luminal breast cancer
969,MONDO:0004990,breast tumor luminal A or B,skos:narrowMatch,DOID:0060548,luminal breast carcinoma A,semapv:LexicalMatching,oaklib,0.800000,oio:hasExactSynonym,oio:hasBroadSynonym,breast tumor luminal
44371,MONDO:0003022,pediatric angiosarcoma,skos:narrowMatch,DOID:0001816,angiosarcoma,semapv:LexicalMatching,oaklib,0.800000,rdfs:label,oio:hasBroadSynonym,angiosarcoma
44372,MONDO:0016982,angiosarcoma,skos:exactMatch,DOID:0001816,angiosarcoma,semapv:LexicalMatching,oaklib,0.941176,oio:hasDbXref,oio:hasDbXref,mesh:d006394
44373,MONDO:0016982,angiosarcoma,skos:exactMatch,DOID:0001816,angiosarcoma,semapv:LexicalMatching,oaklib,0.800000,rdfs:label,oio:hasExactSynonym,angiosarcoma


In [16]:
%%time
# comparison_ms_diff = compare_dataframes(msdf_mondo.df, msdf_lex.df)
# comparison_df = comparison_ms_diff.combined_dataframe
# comparison_df['comment'] = comparison_df['comment'].str.replace('UNIQUE_1', "MONDO_MAPPINGS")
# comparison_df['comment'] = comparison_df['comment'].str.replace('UNIQUE_2', "LEXMATCH")
# comparison_df.head()

# Comparisons
icd_comparison_df = compare_and_comment_df(mondo_icd_df, mondo_icd_lex_df)
omim_comparison_df = compare_and_comment_df(mondo_omim_df, mondo_omim_lex_df)
ordo_comparison_df = compare_and_comment_df(mondo_ordo_df, mondo_ordo_lex_df)
doid_comparison_df = compare_and_comment_df(mondo_doid_df, mondo_doid_lex_df)


CPU times: user 1min 20s, sys: 206 ms, total: 1min 20s
Wall time: 1min 20s


In [17]:
icd_comparison_df
omim_comparison_df
ordo_comparison_df
doid_comparison_df

,subject_id,subject_label,predicate_id,object_id,mapping_justification,object_label,comment,mapping_tool,confidence,subject_match_field,object_match_field,match_string
47923,MONDO:0014278,immunodeficiency 18,skos:exactMatch,DOID:0060017,Unspecified,CD3epsilon deficiency,MONDO_MAPPINGS,NaN,NaN,NaN,NaN,NaN
1245,MONDO:0000790,obsolete Atlantic salmon allergy,skos:exactMatch,DOID:0060515,Unspecified,Atlantic salmon allergy,MONDO_MAPPINGS,NaN,NaN,NaN,NaN,NaN
9957,MONDO:0003607,neuritis of upper limb,skos:exactMatch,DOID:572,Unspecified,mononeuritis of upper limb and mononeuritis mu...,MONDO_MAPPINGS,NaN,NaN,NaN,NaN,NaN
58441,MONDO:0018251,obsolete glycogen storage disease due to phosp...,skos:exactMatch,DOID:0050594,Unspecified,glycogen storage disease IX,MONDO_MAPPINGS,NaN,NaN,NaN,NaN,NaN
65462,MONDO:0021103,obsolete collagen diseases,skos:exactMatch,DOID:854,Unspecified,collagen disease,MONDO_MAPPINGS,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
46157,MONDO:0020290,atrioventricular septal defect,skos:exactMatch,DOID:0050651,semapv:LexicalMatching,atrioventricular septal defect,COMMON_TO_BOTH,oaklib,0.941176,oio:hasDbXref,oio:hasDbXref,omim:600309
46158,MONDO:0020290,atrioventricular septal defect,skos:exactMatch,DOID:0050651,semapv:LexicalMatching,atrioventricular septal defect,COMMON_TO_BOTH,oaklib,0.800000,rdfs:label,oio:hasExactSynonym,atrioventricular septal defect
46159,MONDO:0020290,atrioventricular septal defect,skos:exactMatch,DOID:0050651,semapv:LexicalMatching,atrioventricular septal defect,COMMON_TO_BOTH,oaklib,0.800000,oio:hasExactSynonym,oio:hasExactSynonym,ecd
46160,MONDO:0020290,atrioventricular septal defect,skos:exactMatch,DOID:0050651,semapv:LexicalMatching,atrioventricular septal defect,COMMON_TO_BOTH,oaklib,0.849779,rdfs:label,rdfs:label,atrioventricular septal defect


In [18]:
icd_comparison_df['comment'].drop_duplicates()

51133     MONDO_MAPPINGS
144371          LEXMATCH
65951     COMMON_TO_BOTH
Name: comment, dtype: object

### Split into unmapped dataframes

In [19]:
unmapped_icd_df = get_unmapped_df(icd_comparison_df)
unmapped_omim_df = get_unmapped_df(omim_comparison_df)
unmapped_ordo_df = get_unmapped_df(ordo_comparison_df)
unmapped_doid_df = get_unmapped_df(doid_comparison_df)

unmapped_icd_df.head()


,subject_id,subject_label,predicate_id,object_id,mapping_justification,object_label,comment,mapping_tool,confidence,subject_match_field,object_match_field,match_string
144371,MONDO:0024333,sciatica,skos:exactMatch,ICD10CM:M54.3,semapv:LexicalMatching,Sciatica,LEXMATCH,oaklib,0.849779,rdfs:label,rdfs:label,sciatica
144799,MONDO:0016581,conotruncal heart malformations,skos:relatedMatch,ICD10CM:Q20.0,semapv:LexicalMatching,Common arterial trunk,LEXMATCH,oaklib,NaN,oio:hasExactSynonym,oio:hasRelatedSynonym,persistent truncus arteriosus
143052,MONDO:0001588,chronic lacrimal gland enlargement,skos:exactMatch,ICD10CM:H04.03,semapv:LexicalMatching,Chronic enlargement of lacrimal gland,LEXMATCH,oaklib,0.800000,rdfs:label,oio:hasExactSynonym,chronic enlargement of lacrimal gland
140865,MONDO:0006042,meningeal tuberculosis,skos:exactMatch,ICD10CM:A17.0,semapv:LexicalMatching,Tuberculous meningitis,LEXMATCH,oaklib,0.800000,rdfs:label,oio:hasExactSynonym,tuberculous meningitis
144574,MONDO:0001172,salpingo-oophoritis,skos:exactMatch,ICD10CM:N70,semapv:LexicalMatching,Salpingitis and oophoritis,LEXMATCH,oaklib,0.800000,oio:hasExactSynonym,oio:hasExactSynonym,tubo-ovarian inflammatory disease


In [20]:
print_prefixes(unmapped_doid_df)

subject_prefixes:
 73903    MONDO
Name: subject_id, dtype: object 
           object_prefixes:
 73903    DOID
Name: object_id, dtype: object 
           predicate_ids:
 73903       skos:exactMatch
111908    skos:relatedMatch
112125      skos:broadMatch
107458     skos:narrowMatch
Name: predicate_id, dtype: object 


In [21]:
# Let reviewers check if this makes sense or no.
export_unmatched_exact(unmapped_icd_df, "LEXMATCH", "unmapped_icd_lex.tsv")
export_unmatched_exact(unmapped_omim_df, "LEXMATCH", "unmapped_omim_lex.tsv")
export_unmatched_exact(unmapped_ordo_df, "LEXMATCH", "unmapped_ordo_lex.tsv")
export_unmatched_exact(unmapped_doid_df, "LEXMATCH", "unmapped_doid_lex.tsv")


,subject_id,subject_label,predicate_id,object_id,mapping_justification,object_label,comment,mapping_tool,confidence,subject_match_field,object_match_field,match_string
73903,MONDO:0016642,meningioma,skos:exactMatch,DOID:0080842,semapv:LexicalMatching,intracranial meningioma,LEXMATCH,oaklib,0.941176,oio:hasDbXref,oio:hasDbXref,mesh:d008579
73904,MONDO:0016642,meningioma,skos:exactMatch,DOID:0080842,semapv:LexicalMatching,intracranial meningioma,LEXMATCH,oaklib,0.800000,rdfs:label,oio:hasExactSynonym,intracranial meningioma
111907,MONDO:0026767,"immunodeficiency 74, COVID-19-related, X-linked",skos:exactMatch,DOID:0112063,semapv:LexicalMatching,X-Linked immunodeficiency 74,LEXMATCH,oaklib,0.941176,oio:hasDbXref,oio:hasDbXref,omim:301051
112364,MONDO:0011792,thyroid dyshormonogenesis 6,skos:exactMatch,DOID:0112189,semapv:LexicalMatching,thyroid dyshormonogenesis 6,LEXMATCH,oaklib,0.849779,rdfs:label,rdfs:label,thyroid dyshormonogenesis 6
112365,MONDO:0011792,thyroid dyshormonogenesis 6,skos:exactMatch,DOID:0112189,semapv:LexicalMatching,thyroid dyshormonogenesis 6,LEXMATCH,oaklib,0.800000,rdfs:label,oio:hasExactSynonym,thyroid dyshormonogenesis 6


In [22]:
# Inspect why these are missing from SSSOM mappings
export_unmatched_exact(unmapped_icd_df, "MONDO_MAPPINGS", "unmapped_icd_mondo.tsv")
export_unmatched_exact(unmapped_omim_df, "MONDO_MAPPINGS", "unmapped_omim_mondo.tsv")
export_unmatched_exact(unmapped_ordo_df, "MONDO_MAPPINGS", "unmapped_ordo_mondo.tsv")
export_unmatched_exact(unmapped_doid_df, "MONDO_MAPPINGS", "unmapped_doid_mondo.tsv")

,subject_id,subject_label,predicate_id,object_id,mapping_justification,object_label,comment,mapping_tool,confidence,subject_match_field,object_match_field,match_string
47923,MONDO:0014278,immunodeficiency 18,skos:exactMatch,DOID:0060017,Unspecified,CD3epsilon deficiency,MONDO_MAPPINGS,NaN,NaN,NaN,NaN,NaN
1245,MONDO:0000790,obsolete Atlantic salmon allergy,skos:exactMatch,DOID:0060515,Unspecified,Atlantic salmon allergy,MONDO_MAPPINGS,NaN,NaN,NaN,NaN,NaN
9957,MONDO:0003607,neuritis of upper limb,skos:exactMatch,DOID:572,Unspecified,mononeuritis of upper limb and mononeuritis mu...,MONDO_MAPPINGS,NaN,NaN,NaN,NaN,NaN
58441,MONDO:0018251,obsolete glycogen storage disease due to phosp...,skos:exactMatch,DOID:0050594,Unspecified,glycogen storage disease IX,MONDO_MAPPINGS,NaN,NaN,NaN,NaN,NaN
71940,MONDO:0100471,vitamin D deficiency,skos:exactMatch,DOID:10574,Unspecified,obsolete vitamin D deficiency,MONDO_MAPPINGS,NaN,NaN,NaN,NaN,NaN


In [23]:
# # Add distances
# # Add Levenshtein distance [ 0: Perfect match]
# add_distance(unmapped_mondo_exact, "levenshtein_dist", textdistance.levenshtein.distance)
# # Add Jaccard Index [ 0: Perfect match]
# add_distance(unmapped_mondo_exact, "jaccard_index", textdistance.jaccard.distance)
# # Add Monge-Elkan Distance [ 0: Perfect match]
# add_distance(unmapped_mondo_exact, "monge_elkan", textdistance.monge_elkan.distance)
# unmapped_mondo_exact.to_csv(join(dir_name, "unmapped_mondo_exact.tsv"), sep='\t', index = False)
# unmapped_mondo_exact.head()


In [24]:
combined_msdf_icd = make_msdf(icd_comparison_df, msdf_lex.prefix_map, msdf_lex.metadata)
combined_msdf_omim = make_msdf(omim_comparison_df, msdf_lex.prefix_map, msdf_lex.metadata)
combined_msdf_ordo = make_msdf(ordo_comparison_df, msdf_lex.prefix_map, msdf_lex.metadata)
combined_msdf_doid = make_msdf(doid_comparison_df, msdf_lex.prefix_map, msdf_lex.metadata)

combined_msdf_icd.df.head()

,subject_id,subject_label,predicate_id,object_id,mapping_justification,object_label,comment,mapping_tool,confidence,subject_match_field,object_match_field,match_string
51133,MONDO:0015420,cleft lip and alveolus,skos:broadMatch,ICD10CM:Q36.0,Unspecified,"Cleft lip, bilateral",MONDO_MAPPINGS,NaN,NaN,NaN,NaN,NaN
53434,MONDO:0016276,high-grade neuroendocrine carcinoma of the cer...,skos:broadMatch,ICD10CM:C53.1,Unspecified,Malignant neoplasm of exocervix,MONDO_MAPPINGS,NaN,NaN,NaN,NaN,NaN
60246,MONDO:0018936,osteoblastoma,skos:broadMatch,ICD10CM:D16.6,Unspecified,Benign neoplasm of vertebral column,MONDO_MAPPINGS,NaN,NaN,NaN,NaN,NaN
63728,MONDO:0020104,obsolete rare constitutional hemolytic anemia ...,skos:broadMatch,ICD10CM:D55.3,Unspecified,Anemia due to disorders of nucleotide metabolism,MONDO_MAPPINGS,NaN,NaN,NaN,NaN,NaN
14701,MONDO:0005109,HIV infectious disease,skos:exactMatch,ICD10CM:B20-B20,Unspecified,Human immunodeficiency virus [HIV] disease (B20),MONDO_MAPPINGS,NaN,NaN,NaN,NaN,NaN


In [25]:
# %%time
# df_dict = split_dataframe(combined_msdf)

In [26]:
# mondo_icd_list = [x for x in list(df_dict.keys()) if 'mondo' in x and "icd10" in x]
# mondo_icd_list

In [27]:
# for match in mondo_icd_list:
#     fn = match + ".tsv"
#     df_dict[match].df.to_csv(join(dir_name,fn), sep='\t', index = False)

In [28]:
# df_dict['mondo_exactmatch_icd10cm'].df